# In this .ipnb, we automatically traverse the data table showing the overall death cause ranking system across 51 states based on gender, age.

##  1. Import required packages

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
from pandas import ExcelWriter

## 2. State Name List

In [2]:
def state_list():
    '''
    rtype: dict{state name: state index} 
    
    this function finds out all states plus the US itself name and index from website
    '''
    url = "https://www.worldlifeexpectancy.com/usa-cause-of-death-by-age-and-gender"
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, "html.parser") 
    s_list = soup.find_all("div",class_="scrolling-content-wrapper")

    state_list , state_index= {},0
    for i in s_list[0].find_all('a'):
        print
        state_list[i.text] = str(state_index) if state_index!=0 else ''
        state_index+=1
    #print(state_list)
    
    return state_list

In [4]:
state_list=state_list()

## 3. Collect Data according to state and gender

In [8]:
'''
Given state name and index, this cell can change url accordingly
Save the dataframe into .csv file
'''
gender =['female','male','both']

index = 0
for region, _ in state_list.items():
    writer = pd.ExcelWriter(f'data/newStates/{region}.xlsx')
    state='' if region=='United States' else str(index)
    
    for g in gender:
        # change url to the table we want to collect data now
        url = 'https://www.worldlifeexpectancy.com/j/state-gbd-cause-age?sel=d_35_44&sex={}&state={}'.format(g,state)  
        response = requests.get(url)
        html_doc = response.content 
        json_parsed = json.loads(html_doc)
        
        
        # avoid missing number in url, check whether url exists
        while (json_parsed['chart']['countries']['countryitem'])==[]:
            index+=1
            state='' if region=='United States' else str(index)
            url = 'https://www.worldlifeexpectancy.com/j/state-gbd-cause-age?sel=d_35_44&sex={}&state={}'.format(g,state)  
            response = requests.get(url)
            html_doc = response.content 
            json_parsed = json.loads(html_doc)
        
        # put data into container
        data=defaultdict(list)
        for i in json_parsed['chart']['countries']['countryitem']:
            for j in i.keys():
                data[j].append(i[j])

        
        data_contain ={'name': data['name'], 'rank': data['r_d'], 'death': data['d'], 'rank_0_14': data['r_d_0_14'], 'death_0_14':data['d_0_14'], 'rank_15_24': data['r_d_15_24'],
                       'death_15_24': data['d_15_24'], 'rank_25_34': data['r_d_25_34'], 'death_25_34': data['d_25_34'], 'rank_35_44': data['r_d_35_44'], 'death_35_44': data['d_35_44'], 
                       'rank_45_54': data['r_d_45_54'],'death_45_54': data['d_45_54'], 'rank_55_64': data['r_d_55_64'], 'death_55_64': data['d_55_64'], 'rank_65_74': data['r_d_65_74'], 
                       'death_65_74': data['d_65_74'], 'rank_75': data['r_d_75'], 'death_75': data['d_75']}
        
        # change format to panda Dataframe
        frame = pd.DataFrame(data_contain)  

        
        # write into file
        frame.to_excel(writer, sheet_name=g, index=False,engine='openpyx1')
        print('State: %s    finish --> %s   %s' % (state,g,region))

        
    index+=1
    writer.save()

State:     finish --> female   United States
State:     finish --> male   United States
State:     finish --> both   United States
State: 1    finish --> female   Alabama
State: 1    finish --> male   Alabama
State: 1    finish --> both   Alabama
State: 2    finish --> female   Alaska
State: 2    finish --> male   Alaska
State: 2    finish --> both   Alaska
State: 4    finish --> female   Arizona
State: 4    finish --> male   Arizona
State: 4    finish --> both   Arizona
State: 5    finish --> female   Arkansas
State: 5    finish --> male   Arkansas
State: 5    finish --> both   Arkansas
State: 6    finish --> female   California
State: 6    finish --> male   California
State: 6    finish --> both   California
State: 8    finish --> female   Colorado
State: 8    finish --> male   Colorado
State: 8    finish --> both   Colorado
State: 9    finish --> female   Connecticut
State: 9    finish --> male   Connecticut
State: 9    finish --> both   Connecticut
State: 10    finish --> female   